In [1]:
from scipy import signal
import numpy as np

import matplotlib
matplotlib.use('nbagg')
from pylab import rcParams
rcParams['figure.figsize'] = 9, 4
import matplotlib.pyplot as plt
import scipy

from scipy.fftpack import fft, ifft
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
import copy

from pyha import Hardware, simulate, sims_close, Complex, resize, hardware_sims_equal
from pathlib import Path
from data import load_iq
sys.path.insert(0,'/home/gaspar/git/pyhacores/under_construction/fft/')
from under_construction.fft.bit_reversal_fftshift import BitReversal
from under_construction.fft.conjmult import ConjMult
from under_construction.fft.fft_core import R2SDF
from under_construction.fft.windower import Windower
from under_construction.fft.avg_decimate import AvgDecimate
from under_construction.fft.spectrogram import Spectrogram
from under_construction.fft.bitreversal_fftshift_decimate import BitreversalFFTshiftDecimate
from under_construction.fft.packager import Packager, DataWithIndex, unpackage, package

def imshow(im):
    from skimage.exposure import exposure
    p2, p98 = np.percentile(im, (2, 98))
    im = exposure.rescale_intensity(im, in_range=(p2, p98))

    
    plt.imshow(im, interpolation='nearest', aspect='auto', origin='lower')
    plt.tight_layout()
    plt.show()
    
    
def snr(pure, noisy):
    sig_pow = np.mean(np.abs(pure))
    error = np.array(pure) - np.array(noisy)
    err_pow = np.mean(np.abs(error))
    
    snr_db = 20*np.log10(sig_pow/err_pow)
    return snr_db


# Config

In [2]:
fft_points = 256
freq_axis_avg_decimation = 2
sims = ['MODEL', 'PYHA']
file = '/run/media/gaspar/maxtor/measurement 13.03.2018/mavic_tele/qdetector_20180313122024455464_far_10m_regular/1520936452.2426_fs=20000000.0_bw=20000000.0_fc=2431000000.0_d=0_g=033000.raw'
orig_inp = load_iq(file)
orig_inp = orig_inp[:len(orig_inp)//8]

# Prepare input

In [3]:
oo = np.array([complex(Complex(x, 0, -17, round_style='round')) for x in orig_inp])  # Quantize the inputs!

In [4]:
print(orig_inp[0], oo[0])

(0.0004140752+0.0013231505j) (0.0004119873046875+0.00131988525390625j)


In [5]:
snr(orig_inp, oo)

48.87188083752847

In [6]:
_, _, spectro_out = signal.spectrogram(oo, 1, nperseg=fft_points, return_onesided=False, detrend=False,
                               noverlap=0, window='hann')

# fftshift
spectro_out = np.roll(spectro_out, fft_points//2, axis=0)

# avg decimation
x = np.split(spectro_out, len(spectro_out) // freq_axis_avg_decimation)
golden_output = np.average(x, axis=1)

imshow(golden_output)

<IPython.core.display.Javascript object>